# Welcome to Lumigator Foxfooding from [Mozilla.ai](https://www.mozilla.ai/)! 🐊 🦊

## Agenda

+ Working with Jupyter Notebooks
+ Lumigator Platform Overview  🐊
+ Understanding Machine Learning Workflows 
+ Thunderbird Dataset Walkthrough
+ Explanation of and Examination of Thunderbird Ground Truth
+ Model Selection ( 1 encoder/decoder), (2 decoder), eval against GPT4
+ Run experiment and show results
+ Evaluate results and discuss

## Jupyter Walkthrough

[Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/) are an executable code/text environment for (usually) Python code. Our Jupyter environment is in JupyterHub. To work with Jupyter, click "run cell" to run the code and see results below the cell you're currently running. Cells are executed sequentially. 

In some cells, you will see cases where there are variables that you'll need to pre-populate before running the cell. They look like this. The code will not work unless you replace it!  

```python
# suggestion: "lumigator_enthusiasts"
team_name = TEAM_NAME_HERE
```

# Running cells 
To run a cell, press the "play" icon in the top bar (you can also hit Shift+Enter to run and proceed to the following cell).


<img src="running.png" alt="drawing" width="400"/>

Your files are located on the left-hand side. They'll be saved for the duration of our session, but if you'd like to keep them, make sure to download them. 

<img src="files.png" alt="drawing" width="400"/>


In [1]:
## Lets' try running some code!

print("Welcome to Lumigator!🐊")

# You can see the output below!

Welcome to Lumigator!🐊


For more on notebooks and how cell works, check out this demo. [You can click links in cells.](https://github.com/nbgallery/Jupyter4Analysts/blob/main/J4A%20Notebook%201%20-%20Jupyter%20Syntax%20and%20Other%20Things.ipynb)

## Glossary of terms 

Some terms you'll hear us using throughout the session: 

+ **Machine learning** - The process of creating a model that learns from data
+ **Dataset** - Data used to train models
+ **LLM** - Large language model, [a text-based model that performs next-word predictions](https://www.nvidia.com/en-us/glossary/large-language-models/) 
+ **Tokens** - Words broken up into pieces to be used in an LLM 
+ **Inference** - The process of getting a prediction from a large language model 
+ **Embeddings** - Numerical representations of text generated by modeling 
+ **Encoder-decoder models** - a neural network architecture comprised of two neural networks, an encoder that takes the input vectors from our data and creates an embedding of a fixed length, and a decoder, also a neural network, which takes the embeddings encoded as input and generates a static set of outputs such as translated text or a text summary
+ **Decoder-only models** - Receive a prompt of text directly and predict the next word
+ **Task** - Machine learning tasks to fit a specific model type, including translation, summarization, completion, etc. 
+ **Ground truth** - A dataset that has been evaluated to be true by humans (or LLMs, in some cases) to be correct, that we can use as a point of comparison for our model. 

# Machine Learning Workflows

In machine learning, we are looking to generate a model artifact from data. We have several stages we care about: the data preprocessing, model training, model generation, inference, and evaluation. 

<img src="ml_workflow.png" alt="drawing" width="400"/>

Within the universe of modeling approaches, there are supervised and unsupervised approaches, as well as reinforcement learning. When we think of language modeling, that falls in the realm of neural network approaches. 


<img src="ml_family.png" alt="drawing" width="400"/>


Lumigator focuses on **inference** and **evaluation** for large language models: we want to be able to take our own dataset, perform inference on it, and evaluate the results to see if the model we would like to use produces good results for our use-cases. Use-cases include cases that are specific to our business. 


In order to select an LLM, we need the following stages: 

1. Generate ground truth for our business use-case
2. Pick several models we'd like to use to evaluate
3. Run an evaluation loop consisting of looking at the ground truth in comparison to model results
4. Analyze our evaluations. 

These are the steps that Lumigator completes. Here's a platform overview


<img src="platform.png" alt="drawing" width="400"/>


## Machine learning is alchemy

When we think of traditional software application workflows, we think of an example such as adding a button. We can clearly test that we've added a blue button to our application, and that it works correctly. Machine learning is not like this! It involves a lot of experimentation, tweaking of hyperparameters and prompts and trying different models. Expect for the process to be imperfect, with many iterative loops. Luckily, Lumigator helps take away the uncertainty of at least model selection :)

> There’s a self-congratulatory feeling in the air. We say things like “machine learning is the new electricity”. I’d like to offer an alternative metaphor: machine learning has become alchemy. - [Ben Recht and Ali Rahimi](https://archives.argmin.net/2017/12/05/kitchen-sinks/)

Ultimately, the final conclusion of whether a model is good is if humans think it's good. 

With that in mind, let's dive into setting up experiments with Lumigator to test our models!

In [12]:
# We have a library of utility functions that will help us connect to the Lumigator API
# Let's take a second to walk through them 

import lumigator_demo as ld


In [8]:
# Importing packages we need to work with data 
# python standard libraries
import time

import pandas as pd
import matplotlib.pyplot as plt
import os

# third-party libraries
from datasets import load_dataset
from IPython.display import clear_output

# wrap columns for inspection
pd.set_option('display.max_colwidth', 0)
# stylesheet for visibility
plt.style.use("fast")

# Understanding the Lumigator App and API 

 The app itself consists of an API, which you can access and test out methods with in the [OpenAPI spec](https://swagger.io/specification/), at the platform URL, under docs. 

<img src="openapi.png" alt="drawing" width="200"/>

[Here](https://lumigator.mzai.dev/docs) are the docs for the Lumigator API. In looking at them, we can see that we have 7 endpoints. 
The application is split up into `jobs`, `deployments`, `datasets`, `experiments`, and `completions`.

+ `Datasets` - Data that we add to the platform for evaluation. We can upload, delete, and save different data in the platform. 
+ `Experiments` - Our actual evaluation experiments. We can list all previous evaluations, create new ones, and get their results.
+ `Jobs` - Check running status of lm-buddy evaluation jobs
+ `Deployments` - Running Ray-serve deployments with locally-hosted models
+ `Completions` - Access to external APIs such as Mistral and OpenAI
+ `Health` - Status of the application, jobs and deployments. 




# Experiments
Let's start by creating a team name for our experiments to organize our data, pick a team name below and run the cell. 

In [ ]:
# suggestion: "lumigator_enthusiasts"
team_name = TEAM_NAME_HERE

## Model Task

The task we'll be working with is summarization, aka we want to generate a summary of our text. In this particular case, emails. Finding a good model for summarization is a daunting task, as the typical intuition that larger parameter models generally perform better goes out the window. For summarization, we need to consider the input, which will likely be of a longer context size, and finding models that efficiently deal with those longer contexts is of paramount importance. In our business case, which is to create summaries of conversation threads, much as you might see in Slack or an email chain, the models need to be able to extract key information from those threads while still being able to accept a large context window to capture the entire conversation history.We identified that it is far more valuable to conduct abstractive summaries, or summaries that identify important sections in the text and generate highlights,  rather than extractive ones, which pick a subset of sentences and add them together for our use cases since the final interface will be natural language. We want the summary results not to need to be interpreted from often incoherent text snippets produced by extractive summaries. 

For more on summarization as a use-case, [see our blog post here.](https://blog.mozilla.ai/on-model-selection-for-text-summarization/)

## Ground Truth for Models

In order to generate ground truth, we need to generate ground truth, or a human-generated baseline that we can compare the model against to see how to good it is. The term ground truth comes from geology and geospatial sciences, where actual information was collected on the ground to validate data acquired through remote sensing, such as satellite imagery or aerial photography. Since then, the concept has been adopted in other fields, particularly in machine learning and artificial intelligence, to refer to the accurate, real-world data used for training and testing models. 

 We'll do this by performing inference against existing models that are trained for summarization. Let's take a look at the data we'll be using first. [Generating GT](https://thunderbird.topicbox.com/groups/addons/T18e84db141355abd-M4cca8e3f9e4fee9ae14b9dbb/self-hosted-version-of-extension-is-incorrectly-appearing-in-atn)


## Generating Data for Ground Truth Evaluation

In order to generate a ground truth summary for our data, we first need an input dataset. In this case we use threads from the [Thunderbird public mailing list.](https://thunderbird.topicbox.com/latest)  To generate the ground truth and then later evaluate the model, we need at least 100 samples to start with, where a sample is a single email or single email conversation.

Our selection criteria: 

+ Collect 100 recent and "complete" email threads for evaluation
+ Clean them of email formatting such as `>`
+ One consideration here will be that BART, the baseline model we're using, accepts up to a 1024-token-long context window as input. This means that we have to have input email threads that are ~ approximately 1000 words, so keeping on the conservative side.

Once we've collected them, we'd like to take a look at the data before we generate summaries. 

## How do LLMs work? 

The goal of a transformer model is to take a piece of multimodal content,and learn the latent relationships by creating multiple views of groups of words in the input corpus (multiple context windows). The self-attention mechanism, implemented as scaled dot-product attention in the Transformer paper, creates different context windows of the data a number of times through the six encoder and six decoder layers. The output is the result of the specifimachine learning task — a translated sentence, a summarized paragraph –and the next-to-last layer is the model’s embeddings, which we can use for downstream work.

There are many different kinds of LLMs and many different kinds of architectures. For our stack, we use two different kinds:

+ Encoder/Decoder - BART model
+ Decoder-only - most GPT-family models like Mistral, GPT, and others we'll be working with. 

In [13]:
# show information about the Thunderbird dataset
dataset_id = "db7ff8c2-a255-4d75-915d-77ba73affc53"
r = ld.dataset_info(dataset_id)

{
  "id": "db7ff8c2-a255-4d75-915d-77ba73affc53",
  "filename": "thunderbird_samples.csv",
  "format": "experiment",
  "size": 157184,
  "created_at": "2024-07-17T16:25:37.673229Z"
}


In [14]:
# download the dataset into a pandas dataframe
df = ld.dataset_download(dataset_id)
df.head()

,examples
0,"I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:\n- remove the beta version from ATN,\n- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo you want to self-host, or a single update-repo with the information for all your add-ons)\n- host the XPI files either as an ""beta"" asset in a ""github release"", or directly as a file in the repo (I think the asset is the better choice),\n- make sure that the manifest of your self-hosted XPI points to the correct update.json. John I've removed the beta version of my extension from ATN, and the listing for my extension now shows the current stable release. Thanks for your help!"
1,"the Thunderbird team is preparing the next big release: Thunderbird 128 ESR. Now is a good time to check if your add-ons are compatible. Thunderbird 128 is currently being shipped through the beta release channel. If you have not already installed Thunderbird Beta, you can get it from Thunderbird's download page [1]: Select the desired language, your operating system, and switch the ""release channel"" selection to ""Beta"".\n\nA list of known required changes to make add-ons compatible with Thunderbird 128 can be found on developer.thunderbird.net [2]. These changes mostly affect Experiment add-ons, which directly interact with Thunderbird code. WebExtensions usually do not need updates, but the add-ons team had to introduce the messagesUpdate permission, and browser.messages.update() will stop working, if the new permission has not been requested.\nAnother notably change is the official support of Manifest Version 3 in Thunderbird 128. The add-ons team removed deprecated elements and made additional changes to resolve inconsistencies in the APIs. The full list of changes can be found on webextension-api.thunderbird.net [3].\n\nStarting with Thunderbird 128, the API documentation on webextension-api.thunderbird.net [4] not only includes the WebExtension APIs added by Thunderbird, but also those inherited from Firefox (Thunderbird and Firefox share a significant amount of code). We are listing only methods, which are actually supported and working.\n\nDo not hesitate to reach out for help [5]. Looking forward to see your add-ons running in Thunderbird 128!\n\nJohn\n\n\n[1] : https://www.thunderbird.net/thunderbird/all/\n[2] : https://developer.thunderbird.net/add-ons/updating/tb128\n[3] : https://webextension-api.thunderbird.net/en/128-esr-mv3/changes/esr128.html\n[4] : https://webextension-api.thunderbird.net/en/stable/\n[5] : https://developer.thunderbird.net/add-ons/community"
2,"Hi all, Well I think everything is in the title. We have an extension that shows pdf thumbs in the message pane (faster than openning them). We used to provide PDF.js. Is this still needed? Regards, We usually do not encourage add-ons to depend on files shipped with Thunderbird, because the file could be moved/renamed, which will break your add-on. Ship your own version, so your add-on always uses a known version and stays compatible, even if Thunderbird updates its internal file, which might include API changes which could break your add-on as well. John"
3,"I'm studying how t

In [30]:
# We'd like to make sure our data is clean for LLM input
# This is often not necessary since the model is trained on internet-formatted data
# But we'll be careful here

import re
from string import punctuation

def preprocess_text(text:str):
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    text = re.sub(r"\b[0-9]+\b\s*", "", text)
    return text

df["examples"].map(preprocess_text)

0     i recently added a beta release version 70b1 of my extension clippings for thunderbird and selected the option to selfhost it so that i can make it available to testers separately while regular users continue to see version the current release version however the beta release is now incorrectly appearing in the addons for thunderbird public listing url to atn listing httpsaddonsthunderbirdnetenusthunderbirdaddonclippingstb thanks for the report it is best to file an issue with the addonsserver httpsgithubcomthunderbirdaddonsserver i have not looked at it in detail but selfhosted addons need an updateurl entry in their manifest and that should prevent it from being accepted on atn since we do not sign addons selfhosted addons do not need to be submitted to atn at allmy bestpractice advice is remove the beta version from atn create a dedicated branchrepo which holds the update information either a branch in each addon repo you want to selfhost or a single updaterepo with the inform

In [31]:
# Examine a single sample 
# we define the data with examples
df['examples'].iloc[0]

'I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:\n- remove the beta version from ATN,\n- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo y

In [32]:
# Add a function to do some simple character counts for model input
df['char_count'] = df['examples'].str.len()

In [33]:
df.head

<bound method NDFrame.head of                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
# Show statistics about characters count
df['char_count'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(df['char_count'], bins=30)
ax.set_xlabel('Character Count')
ax.set_ylabel('Frequency')

stats = df['char_count'].describe().apply(lambda x: f"{x:.0f}")

# Add text boxes for statistics
plt.text(1.05, 0.95, stats.to_string(), 
         transform=ax.transAxes, verticalalignment='top')

# Adjust layout
plt.tight_layout()
fig.subplots_adjust(right=0.75)

plt.show()

In [108]:
## Perform Ground Truth Generation with Mistral 

mistral_responses = []

for sample in df['examples'][0:10]:
    res = ld.get_mistral_ground_truth(sample)
    print(f"Mistral Summary:", res)
    mistral_responses.append((sample, res['text']))

Request failed: 500 Server Error: Internal Server Error for url: http://127.0.0.1/api/v1/completions/mistral


HTTPError: 500 Server Error: Internal Server Error for url: http://127.0.0.1/api/v1/completions/mistral

In [42]:
# Let's create a result set we can look at
mistral_results_df = pd.DataFrame(mistral_responses, columns=['original', 'mistral_response'])

mistral_results_df

,original,mistral_response
0,"I recently added a beta release, version 7.0b1, of my extension, Clippings for Thunderbird, and selected the option to self-host it so that I can make it available to testers separately while regular users continue to see version 6.3.5, the current release version. However, the beta release is now incorrectly appearing in the Add-ons for Thunderbird public listing. URL to ATN listing: https://addons.thunderbird.net/en-US/thunderbird/addon/clippings-tb/. Thanks for the report, it is best to file an issue with the addons-server: https://github.com/thunderbird/addons-server I have not looked at it in detail, but self-hosted add-ons need an update_url entry in their manifest and that should prevent it from being accepted on ATN. Since we do not sign add-ons, self-hosted add-ons do not need to be submitted to ATN at all.My best-practice advice is:\n- remove the beta version from ATN,\n- create a dedicated branch/repo which holds the update information (either a branch in each add-on repo you want to self-host, or a single update-repo with the information for all your add-ons)\n- host the XPI files either as an ""beta"" asset in a ""github release"", or directly as a file in the repo (I think the asset is the better choice),\n- make sure that the manifest of your self-hosted XPI points to the correct update.json. John I've removed the beta version of my extension from ATN, and the listing for my extension now shows the current stable release. Thanks for your help!","The user has released a beta version 7.0b1 of their extension, Clippings for Thunderbird, and self-hosted it for testing purposes. However, the beta release appeared in the Add-ons for Thunderbird public listing, which is incorrect. The user was advised to file an issue with the addons-server, as self-hosted add-ons need an 'update_url' entry in their manifest to prevent this. The best practice is to remove the beta version from the ATN listing, create a dedicated branch/repo for update information, host the XPI files as a GitHub release asset or repo file, and ensure the manifest points to the correct update.json. The user has since removed the beta version from ATN, correcting the issue."
1,"the Thunderbird team is preparing the next big release: Thunderbird 128 ESR. Now is a good time to check if your add-ons are compatible. Thunderbird 128 is currently being shipped through the beta release channel. If you have not already installed Thunderbird Beta, you can get it from Thunderbird's download page [1]: Select the desired language, your operating system, and switch the ""release channel"" selection to ""Beta"".\n\nA list of known required changes to make add-ons compatible with Thunderbird 128 can be found on developer.thunderbird.net [2]. These changes mostly affect Experiment add-ons, which directly interact with Thunderbird code. WebExtensions usually do not need updates, but the add-ons team had to introduce the messagesUpdate permission, and browser.messages.update() will stop working, if the new permission has not been requested.\nAnother notably change is the official support of Manifest Version 3 in Thunderbird 128. The add-ons team removed deprecated elements and made additional changes to resolve inconsistencies in the APIs. The full list of changes can be found on webextension-api.thunderbird.net [3].\n\nStarting with Thunderbird 128, the API documentation on webextension-api.thunderbird.net [4] not only includes the WebExtension APIs added by Thunderbird, but also those inherited from Firefox (Thunderbird and Firefox share a significant amount of code). We are listing only methods, which are actually supported and working.\n\nDo not hesitate to reach out for help [5]. Looking forward to see your add-ons running in Thunderbird 128!\n\nJohn\n\n\n[1] : https://www.thunderbird.net/thunderbird/all/\n[2] : https://developer.thunderbird.net/add-ons/updating/tb128\n[3] : https://webextension-api.thunderbird.net/en/128-esr-mv3/changes/

Large language models today are consumed in one of several ways:

+ As API endpoints for proprietary models hosted by OpenAI, Anthropic, or major cloud providers
+ As model artifacts downloaded from HuggingFace’s Model Hub and/or trained/fine-tuned using HuggingFace libraries and hosted on local storage
+ As model artifacts available in a format optimized for local inference, typically GGUF, and accessed via applications like llama.cpp or ollama
+ As ONNX, a format which optimizes sharing between backend ML frameworks

We use API endpoints and local storage in Lumigator. 

In [15]:
# Let's take a look at all available deploys
ld.get_deployments()

{
  "total": 44,
  "items": [
    {
      "id": "bd6e1a72-037e-4b76-ab5c-53adac282b1b",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-29T20:23:49.330389Z",
      "updated_at": null
    },
    {
      "id": "3052d787-0773-4649-9e4a-b93fd31b5056",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-31T16:55:17.888442Z",
      "updated_at": null
    },
    {
      "id": "6dc6f7d8-19cc-4c64-ab2b-40d21b686b36",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-31T17:01:09.772139Z",
      "updated_at": null
    },
    {
      "id": "6ae90573-4555-4c0a-8158-b14eb58665bf",
      "name": "summarizer",
      "description": "Text summarization model",
      "status": "created",
      "created_at": "2024-07-31T17:03:12.373845Z",
      "updated_at": nu

<Response [200]>

In [107]:
## Perform Ground Truth Generation with BART

# UPDATE DEPLOYMENT ID 
deployment_id = "540b155e-40c1-4330-a17f-0eb5b0cc7771"

bart_responses = []

for prompt in df['examples'][0:10]:
    response = ld.get_bart_ground_truth(deployment_id, prompt)
    response_dict = json.loads(response.text)
    results = response_dict.get('deployment_response', {}).get('result', 'No result found')
    bart_responses.append(prompt, results)

Request failed: HTTPConnectionPool(host='127.0.0.1', port=80): Read timed out. (read timeout=10)


ReadTimeout: HTTPConnectionPool(host='127.0.0.1', port=80): Read timed out. (read timeout=10)

In [ ]:
bart_results_df = pd.DataFrame(bart_responses, columns=['original', 'bart_response'])
bart_results_df

In [ ]:
# Combine results and examine multiple versions of ground truth
merged_df = pd.merge(bart_results_df, mistral_results_df, on='original', how='outer')
merged_df.to_csv('ground_truth.csv', index=False)

In [ ]:
merged_df 

In [ ]:
# Now that we have the data, let's save it to the cluster so we can use it later on
ld.dataset_upload("ground_truth.csv")

In [ ]:
# And let's check that data
ld.get_datasets()

## Loading Data

### Loading data
The following dataset is already in the format that we need as input: 
- one field called `examples` containing the text to summarize
- one field called `ground_truth` containing the summaries to the models' outputs against

Note that you can load many different types of file formats in a similar way (see https://huggingface.co/docs/datasets/loading#local-and-remote-files)

In [ ]:
#TODO:

huggingface datasets versus csv
and lm-buddy prefixes

## Dataset Upload

In [ ]:
dataset_name = "thunderbird.csv"
dataset_id = "f5d54efa-247d-4910-9393-f6003da9fb68" # thunderbird pre-saved dataset HuggingFace

r = ld.dataset_info(dataset_id)

## Model Selection

What you see below are different lists of models we have already tested for the summarization task.
The `models` variable at the end provides you with a selection, but you can choose any combination of them.

In [ ]:
enc_dec_models = [
    'hf://facebook/bart-large-cnn',
    'hf://mikeadimech/longformer-qmsum-meeting-summarization', 
    'hf://mrm8488/t5-base-finetuned-summarize-news',
    'hf://Falconsai/text_summarization',
]

dec_models = [
    'mistral://open-mistral-7b',
]

gpts = [
    "oai://gpt-4o-mini",
    "oai://gpt-4-turbo",
    "oai://gpt-3.5-turbo-0125"  
]

# TODO: add llamafile

models = [
    enc_dec_models[0], # bart-large-cnn
    dec_models[0], # Mistral-7B-Instruct-v0.3
    gpts[0] # gpt-4o-mini
]

# show selected models

models

In [ ]:
# TODO
Introduce metrics 

## Run Evaluations

In [ ]:
# change the following to 0 to use all samples in the dataset
max_samples = 10

responses = []
for model in models:
    descr = f"Testing {model} summarization model on {dataset_name}"
    responses.append(ld.experiments_submit(model, team_name, descr, dataset_id, max_samples))

In [ ]:
# TODO
Discuss Ray dashboard/show dashboard

### Track evaluation jobs

Run the following to track your evaluation jobs.

- *NOTE*: you won't be able to run other cells while this one is running. However, you can interrupt it whenever you want by clicking on the "stop" button above and run it at a later time.

In [ ]:
job_ids = [ld.get_resource_id(r) for r in responses]

wip = ld.show_experiment_statuses(job_ids)
while wip == True:
    time.sleep(5)
    clear_output()
    wip=ld.show_experiment_statuses(job_ids)

## Show evaluation results

In [ ]:
# after the jobs complete, gather evaluation results
eval_results = []
for job_id in job_ids:
    eval_results.append(ld.experiments_result_download(job_id))

# convert results into a pandas dataframe
eval_table = ld.eval_results_to_table(models, eval_results)

In [ ]:
eval_table

In [ ]:
eval_results[0]

## Evaluation Results

In [ ]:
#TODO

add eval discussion 